# Notification Setup

In [1]:
from pushbullet import PushBullet
from pywebio.input import *
from pywebio.output import *
from pywebio.session import *
import time

In [2]:
import requests
import json

def pushbullet_noti(title, body):

	TOKEN = 'o.RbpaZEZbP7RQtS8DRita8WKM2nSWtT84' 
	msg = {"type": "note", "title": title, "body": body}	
	resp = requests.post('https://api.pushbullet.com/v2/pushes',
						data=json.dumps(msg),
						headers={'Authorization': 'Bearer ' + TOKEN,
								'Content-Type': 'application/json'})
	if resp.status_code != 200: 
		raise Exception('Error', resp.status_code)
	else:
		print('Sending Message to Friends and Relatives')

# Main Code

In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from pygame import mixer
import time
import numpy as np

pygame 2.5.0 (SDL 2.28.0, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
model = tf.keras.models.load_model(r'C:\Users\imadc\Super Folder\modelss\MobileNet.h5')

In [3]:
def preprocess_eye(eye):
    eye = cv2.resize(eye, (80, 80))
    eye = eye / 255.0
    eye = np.expand_dims(eye, axis=0)
    return eye

In [4]:
#Eyes Detecting
cap = cv2.VideoCapture(0)
song = r"C:\Users\imadc\alert.wav"
mixer.init()
alert_sound = mixer.Sound(song)
point= 0
prev_frame_time = 0
sound_flag = False

In [5]:
#AirBag Model 

from tensorflow.keras.preprocessing import image
airbagmodel = tf.keras.models.load_model(r'C:\Users\imadc\Super Folder\modelss\MobileNetAirBagModel.h5')
consecutive_frames = 0
alert_triggered = False

In [6]:
#Yawn

import dlib
import math

predictor_path = r"C:\Users\imadc\Super Folder\shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
face_detector = dlib.get_frontal_face_detector()

yawn = r"C:\Users\imadc\yawn_alert.mp3"
yawn_alert = mixer.Sound(yawn)

consecutive_frames_yawning = 0
yawn_detected = False  
yawn_count = 0
alert_displayed = False

In [8]:
while True:
    ret, frame = cap.read()
    gray = gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    eyes = eye_cascade.detectMultiScale(gray, minNeighbors=7, scaleFactor=1.8)

    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time

    img = cv2.resize(frame, (110, 110))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    predictions = airbagmodel.predict(img, verbose=0)
    print(f'regular view = {predictions[0][0] * 100:.2f} & Open AirBags = {predictions[0][1] * 100:.2f}')

    for face in faces:
        landmarks = predictor(gray, face)

        upper_lip_point = (landmarks.part(51).x, landmarks.part(51).y)
        lower_lip_point = (landmarks.part(57).x, landmarks.part(57).y)

        # distance between the upper and lower lip points
        distance = math.sqrt((lower_lip_point[0] - upper_lip_point[0]) ** 2 +
                             (lower_lip_point[1] - upper_lip_point[1]) ** 2)

        # ines between the upper and lower lip points
        cv2.line(frame, upper_lip_point, lower_lip_point, (0, 0, 255), 2)

        # green circles
        cv2.circle(frame, upper_lip_point, 3, (0, 255, 0), -1)
        cv2.circle(frame, lower_lip_point, 3, (0, 255, 0), -1)

        if distance > 38:
            consecutive_frames_yawning += 1
            if consecutive_frames_yawning >= 8 and not yawn_detected:
                yawn_count += 1
                yawn_detected = True  
        else:
            consecutive_frames_yawning = 0
            yawn_detected = False  

    if yawn_count >= 4 and not alert_displayed:
        alert_displayed = True
        #alert_text = "Yawning Alert! You are yawning."
        #cv2.putText(frame, alert_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)       
        yawn_alert.play()

    if yawn_count >= 4:  
        yawn_count = 0
        yawn_alert.play()
    
    yawn_text = f"Yawns: {yawn_count}"
    distance_text = f"Distance: {distance:.2f}"
    cv2.putText(frame, yawn_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    cv2.putText(frame, distance_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    if predictions[0][1] >= 0.7:
        consecutive_frames += 1
    else:
        consecutive_frames = 0

    if consecutive_frames >= 10 and not alert_triggered:
        print("Air bags are open! Triggering alert.")
        #pushbullet_noti("Alert", "A I had an accident just now")
        alert_triggered = True
    

    for (x, y, w, h) in eyes:
        eye = frame[y:y + h, x:x + w]
        eye = cv2.resize(eye, (80, 80))
        eye = eye / 255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)
        prediction = model.predict(eye, verbose = 0)
        #print("prediction scores for each frame")
        #print(f'Close = {prediction[0][0]} & Open = {prediction[0][1]}')

        # Condition for Close
        if prediction[0][0] > 0.50:
            point += 1
            
        # Condition for Open
        elif prediction[0][1] > 0.70:
            point = point - 1
            if point < 0:
                point = 0
                
    if point > 15:
        cv2.putText(frame, 'Closed', (100, 190), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4, cv2.FILLED)
    else:
        cv2.putText(frame, 'Open', (100, 190), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), 4, cv2.FILLED)               
                
    if point > 15 and not sound_flag:
        alert_sound.play(-1) 
        sound_flag = True
    elif point <= 15:        
        if sound_flag:
            alert_sound.stop()
            sound_flag = False
                
    p_txt = str(point)
    cv2.putText(frame, f'Points: {p_txt}', (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
    cv2.putText(frame, f'FPS: {int(fps)}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        alert_sound.stop()
        break

alert_sound.stop()
mixer.quit()
cap.release()
cv2.destroyAllWindows()

regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.04
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 100.00 & Open AirBags = 0.00
regular view = 79.26 & Open AirBags = 0.02
regular view = 67.55 & Open AirBa